# CSE283 Final

#### **Generate Results**

In [ ]:
from scripts.hyperparameter_optimization import hyperoptimize_model
label = "cancer"
for model in ["logreg", "randomforest", "xgboost"]:
    print(model, label)
    print("\t ENSG00000216184")
    hyperoptimize_model(label, model, features="cancer_1")
    print("\t ENSG00000199165")
    hyperoptimize_model(label, model, features="cancer_2")
    print("\t ENSG00000216184 and ENSG00000199165")
    hyperoptimize_model(label, model, features="cancer_12")
    print("\t 20 PC")
    hyperoptimize_model(label, model, features="pc")
    print("\t all genes")
    hyperoptimize_model(label, model, features="all")
    print("\n")

In [ ]:
from scripts.hyperparameter_optimization import hyperoptimize_model
label = "recurrence"
for model in ["logreg", "randomforest", "xgboost"]:
    print(model, label)
    print("\t PCA on recurrence related genes")
    hyperoptimize_model(label, model, features="recurrence_pc")
    print("\t 20 PC")
    hyperoptimize_model(label, model, features="pc")
    print("\t all genes")
    hyperoptimize_model(label, model, features="all")
    print("\n")

#### **Make Figures**

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import numpy as np
import pandas as pd
from sklearn.metrics import roc_auc_score, roc_curve, auc, average_precision_score, precision_recall_curve, RocCurveDisplay, PrecisionRecallDisplay
import seaborn as sns

results_folder = "results"
models = ["logreg", "randomforest", "xgboost"]
labels = ["cancer", "recurrence"]
model_name = {"randomforest": "Random Forest",
              "logreg": "Logistic Regression",
              "xgboost": "XGBoost",
             }
cancer_features_set = ["all", "cancer_1", "cancer_12", "pc"]
recurrence_features_set = ["all", "recurrence_pc", "pc"]

##### **Violin plot for Cancer**

In [ ]:
def make_violin_plot(results_folder, model, label, features):
    plt.figure()
    for k in range(5):
        te = pd.read_csv(os.path.join(results_folder, f"{model}-{label}-{features}", str(k), f"test-predictions.csv"))
        va = pd.read_csv(os.path.join(results_folder, f"{model}-{label}-{features}", str(k), f"valid-predictions.csv"))
        if k == 0:
            val_preds = va[["y_pred"]]
        else:
            val_preds["y_pred"] += va["y_pred"]
    val_preds["y_pred"] /= 5     
    val_preds["y_true"] = 2
    df = te[["y_true", "y_pred"]].append(val_preds)

    sns.violinplot(x=df["y_true"], y=df["y_pred"], hue=df["y_true"], palette="Blues", legend=False, cut=0)
    plt.xticks([0,1,2], [f"Non {label.capitalize()} Test", f"{label.capitalize()} Test", f"{label.capitalize()} Valid"])
    plt.ylabel("Probability [u]")
    plt.xlabel("Split")
    if features == "cancer_1":
        l = "just \n AC048346.1 exRNA"
    elif features == "cancer_12":
        l = "just \n AC048346.1 exRNA and MIRNLET7A1"
    elif features == "pc":
        l = "20 main PC"
    else:
        l = "all genes"
    plt.title(f"{model_name[model]} using " + l + "\n Validation Dataset")
    plt.savefig(os.path.join(results_folder, f"{model}-{label}-{features}", f"violin.png"), bbox_inches='tight')
    plt.close()

In [ ]:
label = "cancer"
for model in models:
    for features in cancer_features_set:
        if model == "xgboost" and features == "all":
            continue
        make_violin_plot(results_folder, model, label, features)

##### **ROC and PR Curves**

In [ ]:
mean_fpr = np.linspace(0, 1, 100)
mean_recall = np.linspace(0, 1, 100)
RECALL_LABEL = "Recall | True Positive Rate | TP/(TP+FN)"
SENSITIVITY_LABEL = "Sensitivity | True Positive Rate | TP/(TP+FN)"
FALLOUT_LABEL = "1 - Specificity | False Positive Rate | FP/(FP+TN)"
PRECISION_LABEL = "Precision | Positive Predictive Value | TP/(TP+FP)"

def make_curves(results_folder, model, label, split):
    if label == "cancer":
        features_set = cancer_features_set
    else:
        features_set = recurrence_features_set
    # Create ROC Curve Structure
    fig_roc, ax_roc = plt.subplots()
    ax_roc.plot(
        [0, 1],
        [0, 1],
        linestyle="--",
        lw=2,
        color="gray",
        alpha=0.8,
    )  # , label='Chance')
    ax_roc.set(xlim=[-0.05, 1.05], ylim=[-0.05, 1.05])
    if split == "valid":
        ax_roc.set_title(f"Receiver operating characteristic curve \n"
            f"{model_name[model]} Validation Dataset")
    else:
        ax_roc.set_title(f"Receiver operating characteristic curve \n"
            f"{model_name[model]} {split.capitalize()} Dataset")
    plt.ylabel(SENSITIVITY_LABEL)
    plt.xlabel(FALLOUT_LABEL)

    # Create PR Curve Structure
    fig_pr, ax_pr = plt.subplots()
    f_scores = np.linspace(0.2, 0.8, num=4)
    for f_score in f_scores:
        x = np.linspace(0.01, 1)
        y = f_score * x / (2 * x - f_score)
        ax_pr.plot(x[y >= 0], y[y >= 0], color="gray", alpha=0.2)
    ax_pr.set(xlim=[-0.05, 1.05], ylim=[-0.05, 1.05])
    if split == "valid":
        ax_pr.set_title(f"Precision-Recall curve \n"
            f"{model_name[model]} Validation Dataset")
    else:
        ax_pr.set_title(f"Precision-Recall curve \n"
            f"{model_name[model]} {split.capitalize()} Dataset")
    plt.ylabel(PRECISION_LABEL)
    plt.xlabel(RECALL_LABEL)

    for features in features_set:
        if model == "xgboost" and features == "all":
            continue
        if features == "cancer_1":
            l = "AC048346.1"
        elif features == "cancer_12":
            l = "AC048346.1 and MIRNLET7A1"
        elif features == "recurrence_pc":
            l = "20 PC on Recurrence RG"    
        elif features == "pc":
            l = "20 main PC"
        else:
            l = "All genes"  
        tprs = []
        precisions = []
        aurocs = []
        auprcs = []
        for k in range(5):
            predictions = pd.read_csv(os.path.join(results_folder, f"{model}-{label}-{features}", f"{k}", f"{split}-predictions.csv"))
            y_truth = np.array(predictions["y_true"])
            y_pred = np.array(predictions["y_pred"])

            fpr, tpr, thresholds = roc_curve(y_truth, y_pred)
            precision, recall, _ = precision_recall_curve(y_truth, y_pred)

            roc_auc = auc(fpr, tpr)
            viz = RocCurveDisplay(
                fpr=fpr,
                tpr=tpr,
                roc_auc=roc_auc,
                estimator_name=None,
            )
            interp_tpr = np.interp(mean_fpr, viz.fpr, viz.tpr)
            auroc = viz.roc_auc
            interp_tpr[0] = 0.0

            average_precision = average_precision_score(y_truth, y_pred)
            viz = PrecisionRecallDisplay(
                precision=precision,
                recall=recall,
                average_precision=average_precision,
                estimator_name=None,
            )
            interp_precision = np.interp(mean_recall, viz.recall[::-1], viz.precision[::-1])
            aupr = auc(mean_recall, interp_precision)

            tprs.append(interp_tpr)
            aurocs.append(auroc)
            precisions.append(interp_precision)
            auprcs.append(aupr)
        # Plot ROC curve
        mean_tpr = np.mean(tprs, axis=0)
        # mean_tpr[-1] = 1.0
        mean_auroc = np.mean(aurocs)
        std_auroc = np.std(aurocs)
        ax_roc.plot(
            mean_fpr,
            mean_tpr,
            # color=color,
            label=fr"{l} (AuROC = %0.3f $\pm$ %0.3f)" % (mean_auroc, std_auroc),
            lw=1,
            alpha=0.8,
        )
        std_tpr = np.std(tprs, axis=0)
        tprs_upper = np.minimum(mean_tpr + std_tpr, 1)
        tprs_lower = np.maximum(mean_tpr - std_tpr, 0)
        ax_roc.fill_between(
            mean_fpr,
            tprs_lower,
            tprs_upper,
            #color=color,
            alpha=0.2,
        )
        ax_roc.legend(loc="lower right")
        plt.sca(ax_roc)
        plt.savefig(os.path.join(results_folder, f"{model}-{label}-{split}-roc.png"), bbox_inches='tight')
        plt.close()
        
        # Plot PR curve
        mean_precision = np.mean(precisions, axis=0)
        mean_precision[0] = 1.0
        mean_precision[-1] = 0
        mean_auprc = np.mean(auprcs)
        std_auprc = np.std(auprcs)
        ax_pr.plot(
            mean_recall,
            mean_precision,
            # color=color,
            label=fr"{l} (AuPRC = %0.3f $\pm$ %0.3f)" % (mean_auprc, std_auprc),
            lw=1,
            alpha=0.8,
        )
        std_precision = np.std(precisions, axis=0)
        precision_upper = np.minimum(mean_precision + std_precision, 1)
        precision_lower = np.maximum(mean_precision - std_precision, 0)
        ax_pr.fill_between(
            mean_recall,
            precision_lower,
            precision_upper,
            # color=color,
            alpha=0.2,
        )
        if label == "cancer":
            ax_pr.legend(loc="lower left")
        else:
            ax_pr.legend(loc="upper right")
        plt.sca(ax_pr)
        plt.savefig(os.path.join(results_folder, f"{model}-{label}-{split}-pr.png"), bbox_inches='tight')
        plt.close()

In [ ]:
for label in ["cancer", "recurrence"]:
    for split in ["test", "valid"]:
        for model in models:
            make_curves(results_folder, model, label, split)        